<a href="https://colab.research.google.com/github/said-ml/LearningAgencyLabDeyection/blob/master/LearninAgency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install pyspellchecker

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import gc
import torch
import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,DataCollatorWithPadding
import nltk
! pip install datasets
from datasets import Dataset
from glob import glob
import numpy as np
import pandas as pd
import polars as pl
import re
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.special import softmax
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
import lightgbm as lgb
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
class CFG:
    n_splits = 5
    seed = 42
    num_labels = 6

In [ ]:
import numpy as np
import pandas as pd


# reading files
from zipfile import ZipFile

from typing import List
from typing import Union
from typing import Optional

from abc import ABC
from abc import abstractmethod

zip_file_path=r"/content/learning-agency-lab-automated-essay-scoring-2.zip"
#C:\Users\LG\Downloads\Compressed\learning-agency-lab-automated-essay-scoring-2.zip" # change this paths to fit your local file
train_csv_path="train.csv"
test_csv_path="test.csv"
sample_submission_path="sample_submission.csv"


class data(object):

      def __init__(self,
                   random_state:int=42)->None:

          self.random_state=random_state

      def read_data(self,
                    zip_path: list[str],
                    csv_path: List[str],
                    return_df:bool=True)->Union[pd.DataFrame, np.ndarray]:

          assert  zip_path.endswith(".zip")             # make sure that the file is zip file
          with ZipFile(zip_path, "r") as zip_file:
                  assert csv_path.endswith(".csv")     # make sure that the file is a csv file within a zip file
                  # open the csv file inside of zip file
                  with zip_file.open(csv_path) as csv_fie:
                       df=pd.read_csv(csv_fie)
                       if return_df:
                           return df
                       else:
                           np_array=np.array(df)
                           return np_array


      def split(self,
                    data:Union[pd.DataFrame, np.ndarray],
                    n_samples:Optional[int]=None,
                    train_fraction:float=0.75,
                    shuffle:bool=True,
                    random_state:int=42)->None:

                    pass

      def sample_submission(self)->None:
           assert sample_submission_path.endswith(".csv")
           df=self.read_data(zip_file_path, sample_submission_path)
           return df

      def display_data(self,
                        n_samples:int=5,
                        feature_name:str="")->None:
          pass



      def shape(self)->tuple[int]:
          np_array=self.read_data(return_df=False)
          return np_array.shape






data=data()
train=data.read_data(zip_file_path, train_csv_path)
test=data.read_data(zip_file_path, test_csv_path)
print("__________________  Train____________________")
print(train.tail())
print("__________________  Test ____________________")
print(test.tail())


train=train[:2000]




__________________  Train____________________
      essay_id                                          full_text  score
17302  ffd378d  the story " The Challenge of Exploing Venus " ...      2
17303  ffddf1f  Technology has changed a lot of ways that we l...      4
17304  fff016d  If you don't like sitting around all day than ...      2
17305  fffb49b  In "The Challenge of Exporing Venus," the auth...      1
17306  fffed3e  Venus is worthy place to study but dangerous. ...      2
__________________  Test ____________________
  essay_id                                          full_text
0  000d118  Many people have car where they live. The thin...
1  000fe60  I am a scientist at NASA that is discussing th...
2  001ab80  People always wish they had the same technolog...


In [ ]:

import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)


cList = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would",  ## --> he had or he would
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   ## --> I had or I would
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",   ## --> It had or It would
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",   ## --> It had or It would
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
}
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)
def dataPreprocessing(x):
    # Convert words to lowercase
    #x = x.lower()
    # Remove HTML
    #x = removeHTML(x)
    # Delete strings starting with @
    #x = re.sub("@\w+", '',x)
    # Delete Numbers
    #x = re.sub("'\d+", '',x)
    #x = re.sub("\d+", '',x)
    # Delete URL
    #x = re.sub("http\w+", '',x)
    # Remove \xa0
    #x = x.replace(u'\xa0',' ')
    # Replace consecutive empty spaces with a single space character
    #x = re.sub(r"\s+", " ", x)
    #x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    #x = re.sub(r"\.+", ".", x)
    #x = re.sub(r"\,+", ",", x)
#     x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    # Remove empty characters at the beginning and end
    #x = x.strip()
    return x


def dataPreprocessing_w_contract_punct_remove(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    #x = remove_punctuation(x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

In [ ]:
# TODO: can be fixed by keeping "\n" and removed empty paragraph entries
import polars as pl
columns = [(pl.col("full_text").str.split(by="\n\n").alias("paragraph"))]
train = pl.from_pandas(train).with_columns(columns)
test = pl.from_pandas(test).with_columns(columns)

In [ ]:

def Paragraph_Preprocess(tmp):
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp

# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
def Paragraph_Eng(train_tmp):
    aggs = [
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [50,75,100,125,150,175,200,250,300,350,400,500,600,700] ],
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]],
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea],
    ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)

# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(5)

Features Number:  43


,essay_id,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,...,paragraph_word_cnt_sum,paragraph_len_kurtosis,paragraph_sentence_cnt_kurtosis,paragraph_word_cnt_kurtosis,paragraph_len_q1,paragraph_sentence_cnt_q1,paragraph_word_cnt_q1,paragraph_len_q3,paragraph_sentence_cnt_q3,paragraph_word_cnt_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,497,NaN,NaN,NaN,2677.0,14.0,497.0,2677.0,14.0,497.0
1,000fe60,5,5,5,5,5,5,4,3,3,...,332,-1.306843,-1.421811,-1.388460,235.0,4.0,46.0,398.0,6.0,77.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,559,-1.746976,-1.592593,-1.707851,576.0,5.0,101.0,927.0,8.0,165.0
3,001bdc0,5,5,5,5,4,4,4,4,4,...,451,-1.461307,-1.371972,-1.331901,366.0,2.0,65.0,816.0,8.0,129.0
4,002ba53,4,4,4,4,4,4,4,4,4,...,386,-1.465983,-1.616116,-1.398361,29.0,1.0,14.0,559.0,5.0,93.0


In [ ]:
# sentence feature
def Sentence_Preprocess(tmp):
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))
    return tmp

# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [15,50,100,150,200,250,300] ],
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in sentence_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in sentence_fea],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train)

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

Features Number:  68


,essay_id,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,...,sentence_len_last,sentence_word_cnt_last,sentence_len_sum,sentence_word_cnt_sum,sentence_len_kurtosis,sentence_word_cnt_kurtosis,sentence_len_q1,sentence_word_cnt_q1,sentence_len_q3,sentence_word_cnt_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,47,10,2660,505,1.395172,2.240800,115.0,20.0,231.0,39.0
1,000fe60,5,5,5,5,5,5,4,3,3,...,124,25,1650,344,1.115399,0.693618,53.0,13.0,124.0,25.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,58,10,3044,570,-0.420081,0.107472,90.0,17.0,151.0,29.0


In [ ]:
import pandas as pd

data = pd.Series(["This is a string", "Another string"])
words = data.str.split(expand=True)  # No expand argument

print(words)

         0       1     2       3
0     This      is     a  string
1  Another  string  None    None


In [ ]:
# word feature
def Word_Preprocess(tmp):
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)

    return tmp

# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ],
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train)

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

Features Number:  89


,essay_id,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,...,word_12_cnt,word_13_cnt,word_14_cnt,word_15_cnt,word_len_max,word_len_mean,word_len_std,word_len_q1,word_len_q2,word_len_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,7,6,5,2,25,4.423935,2.577418,3.0,4.0,5.0
1,000fe60,5,5,5,5,5,5,4,3,3,...,4,2,0,0,13,4.091463,2.255307,2.0,4.0,5.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,16,12,7,3,19,4.610603,2.699905,3.0,4.0,5.0


In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,3),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])

print("#"*80)
vect_feat_names=vectorizer.get_feature_names_out()
print(vect_feat_names[100:110])
print("#"*80, "\n\n")

# Convert to array
dense_matrix = train_tfid.toarray()

# Convert to dataframe
df = pd.DataFrame(dense_matrix)

# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

################################################################################
['  E' '  E a' '  E l' '  E u' '  E v' '  E x' '  F' '  F a' '  F o'
 '  F r']
################################################################################ 


Features Number:  3406


,essay_id,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,...,tfid_3307,tfid_3308,tfid_3309,tfid_3310,tfid_3311,tfid_3312,tfid_3313,tfid_3314,tfid_3315,tfid_3316
0,000d118,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.034295,0.071334,0.0,0.0
1,000fe60,5,5,5,5,5,5,4,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [ ]:
"""
!pip install nltk
import nltk
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
"""
import nltk

# Download the stopwords list (one-time download)
nltk.download('stopwords')

# Access the downloaded stopwords using nltk.corpus.stopwords
stopwords_list = nltk.corpus.stopwords.words('english')

# TfidfVectorizer parameter
word_vectorizer = TfidfVectorizer(
    strip_accents='ascii',
    analyzer = 'word',
    ngram_range=(1,1),
    min_df=0.15,
    max_df=0.85,
    sublinear_tf=True,
    stop_words=stopwords_list,
)

"""
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
processed_text = train.to_pandas()["full_text"].progress_apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))


import pandas as pd
from tqdm import tqdm


processed_text = (
    train["full_text"].to_pandas()
    .pipe(tqdm.pandas, desc="Preprocessing Text")
)
"""

import pandas as pd
import tqdm

processed_text = []
for text in tqdm.tqdm(train.to_pandas()["full_text"]):
    processed_text.append(dataPreprocessing_w_contract_punct_remove(text))

train_tfid = word_vectorizer.fit_transform(processed_text)
#train_tfid = word_vectorizer.fit_transform([i for i in processed_text])
#train_tfid = word_vectorizer.fit_transform([i for i in processed_text])

# Convert to array
dense_matrix = train_tfid.toarray()
# Convert to dataframe
df = pd.DataFrame(dense_matrix)
# rename features
tfid_w_columns = [ f'tfid_w_{i}' for i in range(len(df.columns))]
df.columns = tfid_w_columns
df['essay_id'] = train_feats['essay_id']

df.head()
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print("Features Number:")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 2000/2000 [00:01<00:00, 1039.31it/s]


Features Number:


In [ ]:
from nltk.tokenize import TreebankWordDetokenizer
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spellchecker import SpellChecker

class Preprocessor:
    def __init__(self) -> None:
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        self.spellchecker = SpellChecker()

    def spelling(self, text):
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))
        return amount_miss

    def count_sym(self, text, sym):
        sym_count = 0
        for l in text:
            if l == sym:
                sym_count += 1
        return sym_count

    def run(self, data: pd.DataFrame, mode:str) -> pd.DataFrame:

        # preprocessing the text
        data["processed_text"] = data["full_text"].apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))

        # Text tokenization
        data["text_tokens"] = data["processed_text"].apply(lambda x: word_tokenize(x))

        # essay length
        data["text_length"] = data["processed_text"].apply(lambda x: len(x))

        # essay word count
        data["word_count"] = data["text_tokens"].apply(lambda x: len(x))

        # essay unique word count
        data["unique_word_count"] = data["text_tokens"].apply(lambda x: len(set(x)))

        # essay sentence count
        data["sentence_count"] = data["full_text"].apply(lambda x: len(x.split('.')))

        # essay paragraph count
        data["paragraph_count"] = data["full_text"].apply(lambda x: len(x.split('\n\n')))

        # count misspelling
        data["splling_err_num"] = data["processed_text"].apply(self.spelling)
        print("Spelling mistake count done")

        # ratio fullstop / text_length ** new
        data["fullstop_ratio"] = data["full_text"].apply(lambda x: x.count(".")/len(x))

        # ratio comma / text_length ** new
        data["comma_ratio"] = data["full_text"].apply(lambda x: x.count(",")/len(x))

        return data

preprocessor = Preprocessor()
tmp = preprocessor.run(train.to_pandas(), mode="train")
train_feats = train_feats.merge(tmp, on='essay_id', how='left')
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Spelling mistake count done
Features Number


In [ ]:
# Paragraph
tmp = Paragraph_Preprocess(test)
test_feats = Paragraph_Eng(tmp)

# Sentence
tmp = Sentence_Preprocess(test)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

# Word
tmp = Word_Preprocess(test)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

# Tfidf
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

# Word Tfidf
processed_text = test.to_pandas()["full_text"].apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))
test_w_tfid = word_vectorizer.transform([i for i in processed_text])
dense_matrix = test_w_tfid.toarray()
df_w = pd.DataFrame(dense_matrix)
tfid_w_columns = [ f'tfid_w_{i}' for i in range(len(df_w.columns))]
df_w.columns = tfid_w_columns
df_w['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df_w, on='essay_id', how='left')

# Extra feature
tmp = preprocessor.run(test.to_pandas(), mode="train")
test_feats = test_feats.merge(tmp, on='essay_id', how='left')

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number:" ')

Spelling mistake count done
Features number:" 


In [ ]:
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
for i, (_, val_index) in enumerate(skf.split(train_feats, train_feats["score"])):
    train_feats.loc[val_index, "fold"] = i
print(train_feats.shape)
# train_feats.head()

(2000, 3527)


In [ ]:
target = "score"
train_drop_columns = ["essay_id", "fold", "full_text", "paragraph", "text_tokens", "processed_text"] + [target]

In [ ]:
train_feats.drop(columns=train_drop_columns).head()

,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,paragraph_350_cnt,...,tfid_w_104,tfid_w_105,text_length,word_count,unique_word_count,sentence_count,paragraph_count,splling_err_num,fullstop_ratio,comma_ratio
0,1,1,1,1,1,1,1,1,1,1,...,0.000000,0.127141,2642,536,224,14,1,52,0.004856,0.005230
1,5,5,5,5,5,5,4,3,3,2,...,0.145599,0.000000,1665,370,151,20,5,34,0.011384,0.003595
2,4,4,4,4,4,4,4,4,4,4,...,0.000000,0.000000,3061,602,230,25,4,42,0.007800,0.005200
3,5,5,5,5,4,4,4,4,4,4,...,0.148112,0.000000,2665,489,218,24,5,43,0.008515,0.007034
4,4,4,4,4,4,4,4,4,4,4,...,0.000000,0.000000,2168,403,143,16,6,35,0.006793,0.005888


In [ ]:
test_drop_columns = ["essay_id", "full_text", "paragraph", "text_tokens", "processed_text"]

In [ ]:
test_feats.drop(columns=test_drop_columns).head()

,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,paragraph_350_cnt,...,tfid_w_104,tfid_w_105,text_length,word_count,unique_word_count,sentence_count,paragraph_count,splling_err_num,fullstop_ratio,comma_ratio
0,1,1,1,1,1,1,1,1,1,1,...,0.000000,0.127141,2642,536,224,14,1,52,0.004856,0.005230
1,5,5,5,5,5,5,4,3,3,2,...,0.145599,0.000000,1665,370,151,20,5,34,0.011384,0.003595
2,4,4,4,4,4,4,4,4,4,4,...,0.000000,0.000000,3061,602,230,25,4,42,0.007800,0.005200


In [ ]:
# idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = (y_true + a).round()
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True

def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

def qwk_param_calc(y):
    a = y.mean()
    b = (y ** 2).mean() - a**2
    return np.round(a, 4), np.round(b, 4)
# a = 2.948
# b = 1.092

In [ ]:
models = []

callbacks = [
    lgb.log_evaluation(period=25),
    lgb.early_stopping(stopping_rounds=75,first_metric_only=True)
]
for fold in range(CFG.n_splits):

    model = lgb.LGBMRegressor(
        objective = qwk_obj, metrics = 'None', learning_rate = 0.1, max_depth = 5,
        num_leaves = 10, colsample_bytree=0.5, reg_alpha = 0.1, reg_lambda = 0.8,
        n_estimators=1024, random_state=CFG.seed, verbosity = - 1
    )

    a, b = qwk_param_calc(train_feats[train_feats["fold"] != fold]["score"])

    # Take out the training and validation sets for 5 kfold segmentation separately
    X_train = train_feats[train_feats["fold"] != fold].drop(columns=train_drop_columns)
    y_train = train_feats[train_feats["fold"] != fold]["score"] - a

    X_eval = train_feats[train_feats["fold"] == fold].drop(columns=train_drop_columns)
    y_eval = train_feats[train_feats["fold"] == fold]["score"] - a

    print('\nFold_{} Training ================================\n'.format(fold+1))
    print(f"Fold {fold} a: {a}  ;;  b: {b}")
    # Training model
    lgb_model = model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_eval, y_eval)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks
    )
    models.append(model)


Fold_1 Training ================================

Fold 0 a: 2.9481  ;;  b: 1.0492
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.824972	valid's QWK: 0.663114
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	train's QWK: 0.890948	valid's QWK: 0.695276
[75]	train's QWK: 0.931791	valid's QWK: 0.702468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	train's QWK: 0.965811	valid's QWK: 0.697364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[125]	train's QWK: 0.980972	valid's QWK: 0.700601
[LightGBM] [Warning] No furt

In [ ]:
preds, trues = [], []

for fold, model in enumerate(models):
    X_eval_cv = train_feats[train_feats["fold"] == fold].drop(columns=train_drop_columns)
    y_eval_cv = train_feats[train_feats["fold"] == fold]["score"]

    pred = model.predict(X_eval_cv) + a

    trues.extend(y_eval_cv)
    preds.extend(np.round(pred, 0))

v_score = cohen_kappa_score(trues, preds, weights="quadratic")

print(f"Validation score : {v_score}")

Validation score : 0.7417726184685081


In [ ]:
# predecting for 5 models
preds = []
for fold, model in enumerate(models):
    X_eval_cv = test_feats.drop(columns=test_drop_columns)
    # pred = model.predict(X_eval_cv)
    pred = model.predict(X_eval_cv) + a
    preds.append(pred)

# Combining the 5 model results
for i, pred in enumerate(preds):
    test_feats[f"score_pred_{i}"] = pred
test_feats["score"] = np.round(test_feats[[f"score_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1),0).astype('int32')

In [ ]:
test_feats.head()

,essay_id,paragraph_50_cnt,paragraph_75_cnt,paragraph_100_cnt,paragraph_125_cnt,paragraph_150_cnt,paragraph_175_cnt,paragraph_200_cnt,paragraph_250_cnt,paragraph_300_cnt,...,paragraph_count,splling_err_num,fullstop_ratio,comma_ratio,score_pred_0,score_pred_1,score_pred_2,score_pred_3,score_pred_4,score
0,000d118,1,1,1,1,1,1,1,1,1,...,1,52,0.004856,0.005230,1.795787,2.308993,2.619333,2.335459,2.374532,2
1,000fe60,5,5,5,5,5,5,4,3,3,...,5,34,0.011384,0.003595,2.891803,2.703045,2.541002,2.995581,2.752867,3
2,001ab80,4,4,4,4,4,4,4,4,4,...,4,42,0.007800,0.005200,4.143882,4.122020,4.052899,4.064608,4.256827,4


In [ ]:
test_feats[["essay_id", "score"]].to_csv("submission.csv", index=False)